To learn more about COCO integration on FiftyOne visit <a href="https://voxel51.com/docs/fiftyone/integrations/coco.html">this page</a>

## Import

In [ ]:
%env FIFTYONE_DATABASE_URI=mongodb://127.0.0.1:27017
import fiftyone as fo

## Create dataset

In [ ]:
DATASET_NAME = "DATASET_NAME"
IMAGES_DIR = "/tmp/bucket/..."
COCO_GT_PATH = "/tmp/bucket/..."

# Remove old dataset if existing
if DATASET_NAME in fo.list_datasets():
    dataset = fo.load_dataset(DATASET_NAME)
    dataset.delete()

# Create dataset
dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    name=DATASET_NAME,
    data_path=IMAGES_DIR,
    label_types=["detections"],  # "detections", "segmentations", "keypoints"
    labels_path=COCO_GT_PATH,
    include_id=True,
)

# Make the dataset persistent
dataset.persistent = True

In [ ]:
for sample in dataset:
    print(sample)
    break

## Import predictions (unfiltered)

In [ ]:
import json

COCO_PREDS_PATH = "/tmp/bucket/..."

with open(COCO_PREDS_PATH) as file:
    predictions = json.load(file)

In [ ]:
dataset.default_classes

In [ ]:
import fiftyone.utils.coco as fouc

# https://voxel51.com/docs/fiftyone/api/fiftyone.utils.coco.html#fiftyone.utils.coco.add_coco_labels
# Add COCO predictions to "predictions" field of dataset
classes = dataset.default_classes
fouc.add_coco_labels(dataset, "predictions", predictions["annotations"], classes, coco_id_field="coco_id")

# Verify that predictions were added
print(dataset.count("predictions"))

## Find optimal thresholds

In [ ]:
# !pip install ipywidgets==7.7.2

In [ ]:
results = dataset.evaluate_detections(
    "predictions",
    gt_field="detections",
    method="coco",
    compute_mAP=True,
)

plot = results.plot_pr_curves(classes=classes[1:], iou_thresh=0.50)

In [ ]:
import numpy as np

opt_threshs = {}
shapes = []

for curve in plot._figure.data:
    name = curve["name"]
    r = curve["x"]
    p = curve["y"]
    t = curve["customdata"]
    curve_name = curve["name"]
    class_name = curve["text"][0]
    f1 = 2 * ((p * r) / (p + r))
    amax = np.argmax(f1)
    opt_thresh = t[amax]
    print("Optimal threshold for {}:\n{:.2f} (with F1={:.2f} / PREC={:.2f} / REC={:.2f})".format(curve_name, opt_thresh, f1[amax], p[amax], r[amax]))
    opt_threshs[class_name] = opt_thresh
    shapes.append(dict(type="circle", x0=r[amax]-0.01, y0=p[amax]-0.01, x1=r[amax]+0.01, y1=p[amax]+0.01, line_color="red"))

In [ ]:
plot.update_layout(
    width=1000,
    height=700,
    autosize=False,
    xaxis_range=[-0.05, 1.05],
    yaxis_range=[-0.05, 1.05],
    shapes=shapes,
)
plot.show()

## Import predictions (filtered)

In [ ]:
import fiftyone.utils.coco as fouc

print("Number of annotations before filtering: {}".format(len(predictions["annotations"])))
predictions_filtered = [
    ann for ann in predictions["annotations"]
    if ann["score"] >= opt_threshs[classes[ann["category_id"]]]
]
print("Number of annotations after filtering: {}".format(len(predictions_filtered)))

classes = dataset.default_classes
fouc.add_coco_labels(dataset, "predictions_filtered", predictions_filtered, classes, coco_id_field="coco_id")

# Verify that predictions were added
print(dataset.count("predictions"))

## Print report

In [ ]:
results = dataset.evaluate_detections(
    "predictions_filtered",
    gt_field="detections",
    method="coco",
    eval_key="eval",
)

results.print_report(classes=classes[1:])

# Print some statistics about the total TP/FP/FN counts
print("TP: %d" % dataset.sum("eval_tp"))
print("FP: %d" % dataset.sum("eval_fp"))
print("FN: %d" % dataset.sum("eval_fn"))

## Print confusion matrix

In [ ]:
results = dataset.evaluate_detections(
    "predictions_filtered",
    gt_field="detections",
    method="coco",
    classwise=False,
)

from matplotlib import pyplot as plt
params = {
    'figure.figsize': (18, 10),
    'font.size': 14,
    'axes.labelsize': 16,
    'axes.titlesize': 18,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
}
plt.rcParams.update(params)
plot = results.plot_confusion_matrix(classes=classes[1:], backend="matplotlib", cmap='Blues', figsize=(18,12))
plt.show()

## Start session

In [ ]:
from IPython.core.display import display, HTML
from uuid import uuid4

session = fo.launch_app(dataset, port=6008, auto=False)

uuid = str(uuid4())
print(uuid)
URL = session.url + f"?subscription={uuid}&polling=true"
display(
    HTML(
        """
        <a href="{}" target="_blank"><h1>Open session in a new tab</h1></a>
        """.format(URL)
    )
)

## Launch in notebook

In [ ]:
# Run this to use the app here
session.show()